## E-Lite model

## Extract an E-lite sector

E-Lite is the current baseline ITER full 360 degree model. Since calculations
with this model are computationally very heavy, it is often useful to extract
one (or more, if needed) of its sector(s), to save computational resources and
time. The Excel spreadsheet of E-Lite envelopes structure is required for the
extraction. A tolerance must be set when extracting a sector: it will define the
minimum distance for planes to be considered equal. It is needed to modify the
lateral periodic boundary planes of the sector(s) and should be set according to
the value in DBCN card in the model (The DBCN card is documented in MCNP manual,
in MCNP6.2 the 9th entry of DBCN card defines "the distance allowed between 
coincident repeated-structures surfaces for them still to be considered 
coincident"). 
The result is an input of the chosen sector(s) bounded by reflective planes.

In [ ]:
from f4enix.input.elite import Elite_Input
import tempfile  # To have a scratch directory for the example
import os

# Define the path to Excel spreadsheet
excel_path = 'E-Lite_Block_Structure_Summary.xlsx'

# Read E-Lite input, this time by using Elite_Input subclass
elite_inp = Elite_Input.from_input('E-Lite.i')

Extract a single sector:
You must provide the sector number, the path of the excel file, the output
file name, the tolerance (default 1e-5).

In [ ]:
elite_inp.extract_sector(1, excel_file=excel_path,
                         outfile=os.path.join(tempfile.gettempdir(), 'sector1.i'))

For NBI, sectors 2 and 3 must be extracted together
You can also change the tolerance and choose to check if the Excel spreadsheet
is up to date with the model, to guarantee the functioning of the method

In [ ]:
elite_inp.extract_sector('2 & 3', excel_file=excel_path,
                         outfile=os.path.join(tempfile.gettempdir(), 'NBI.i'),
                         tol=1e-4, check_Elite=True)

You can also extract more than one sector, by providing their numbers in a
list, in counterclockwise direction. The sectors must be contiguous, e.g. you
cannot extract at the same time sector 4 and 7.

In [ ]:
elite_inp.extract_sector(['2 & 3', 4], excel_file=excel_path, 
                         outfile=os.path.join(tempfile.gettempdir(), 'NBI_sec4.i'),
                         tol=1e-6, check_Elite=False)

To actually use the class Elite_Input you will need the Excel spreadsheet with the block structure of E-lite and the patch to fix the lost particles arising in the model of the sector after the extraction. You can require them by contacting the developers.

## Format materials for E-lite

E-lite has a specific formatting for materials. Generally in F4Enix, for each zaid, a dollar command is added reporting:

* The weight percentage of the element it belongs to in the submaterial
* The atom fraction of the zaid in the element (abundance)

In addition to this, the E-lite standard requires the atom fraction of the single zaids to actually represent the atomic density of such zaid in the material (given a certain nominal density of the material). This will cause the fractions to be not normalized.

There is a specific function in F4Enix to port all materials in an MCNP input (or a single material) to such standard.

First, let's load an example

In [16]:
from f4enix.input.materials import Material, MatCardsList
from f4enix.input.libmanager import LibManager

lm = LibManager()
material = Material.from_zaids([('H', 2), ('O', 1)], lm, '31c')
mat_library = MatCardsList([material])
print(mat_library.to_text())

C 
M1
       1001.31c        1.999710E+0     $ H-1    WEIGHT(%) 11.19 AB(%) 99.986
       1002.31c        2.900000E-4     $ H-2    WEIGHT(%) 11.19 AB(%) 0.0145
       8016.31c        9.975700E-1     $ O-16   WEIGHT(%) 88.81 AB(%) 99.757
       8017.31c        3.835000E-4     $ O-17   WEIGHT(%) 88.81 AB(%) 0.03835
       8018.31c        2.045000E-3     $ O-18   WEIGHT(%) 88.81 AB(%) 0.2045


And then convert it to the E-lite standard

In [17]:
density = 0.98 # g/cm3

mat_library.fractions_to_atom_densities(lm, density)
print(mat_library.to_text())

C 
M1
       1001.31c        6.550811E-2     $ H-1    WEIGHT(%) 11.19 AB(%) 99.986
       1002.31c        9.500054E-6     $ H-2    WEIGHT(%) 11.19 AB(%) 0.0145
       8016.31c        3.267920E-2     $ O-16   WEIGHT(%) 88.81 AB(%) 99.757
       8017.31c        1.256300E-5     $ O-17   WEIGHT(%) 88.81 AB(%) 0.03835
       8018.31c        6.699176E-5     $ O-18   WEIGHT(%) 88.81 AB(%) 0.2045
